# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298485457071                   -0.85    5.1         
  2   -8.300212403157       -2.76       -1.25    1.0    102ms
  3   -8.300435974268       -3.65       -1.89    3.2    140ms
  4   -8.300461267550       -4.60       -2.75    1.8    113ms
  5   -8.300464073401       -5.55       -3.03    3.1    168ms
  6   -8.300464353854       -6.55       -3.21    7.0    240ms
  7   -8.300464494591       -6.85       -3.35    4.4    166ms
  8   -8.300464571930       -7.11       -3.50    2.2    138ms
  9   -8.300464626824       -7.26       -3.72    1.5    125ms
 10   -8.300464629368       -8.59       -3.77    1.0    104ms
 11   -8.300464639916       -7.98       -3.98    1.4    148ms
 12   -8.300464643955       -8.39      

In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67370008249                   -0.70    5.2         
  2   -16.67862047049       -2.31       -1.15    2.0    297ms
  3   -16.67919413799       -3.24       -1.87    2.8    319ms
  4   -16.67927972415       -4.07       -2.74    4.8    394ms
  5   -16.67928591529       -5.21       -3.18    5.0    406ms
  6   -16.67928618563       -6.57       -3.47    2.1    315ms
  7   -16.67928621297       -7.56       -3.89    1.6    262ms
  8   -16.67928622133       -8.08       -4.58    2.8    323ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32400337943                   -0.56    6.9         
  2   -33.33240668881       -2.08       -1.00    1.0    997ms
  3   -33.33406341593       -2.78       -1.74    4.8    1.31s
  4   -33.33578085693       -2.77       -2.30    3.4    1.24s
  5   -33.33691818202       -2.94       -2.54   10.9    1.71s
  6   -33.33693903179       -4.68       -2.93    2.5    1.04s
  7   -33.33694358451       -5.34       -3.69    4.6    1.32s
  8   -33.33694369497       -6.96       -3.86    5.0    1.49s
  9   -33.33694377779       -7.08       -4.46    1.8    1.02s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298384332123                   -0.85    5.1         
  2   -8.300258756016       -2.73       -1.59    1.0   82.7ms
  3   -8.300422561312       -3.79       -2.48    3.1    121ms
  4   -8.300344790605   +   -4.11       -2.21    3.6    141ms
  5   -8.300464120499       -3.92       -3.52    1.1    113ms
  6   -8.300464520630       -6.40       -3.73    4.2    174ms
  7   -8.300464635551       -6.94       -4.18    1.0   95.4ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32617272890                   -0.56    7.0         
  2   -33.23531002893   +   -1.04       -1.23    1.4    860ms
  3   +37.40897752614   +    1.85       -0.14    9.2    2.44s
  4   -33.29422306429        1.85       -1.58    6.5    2.11s
  5   -33.05028480813   +   -0.61       -1.22    3.1    1.33s
  6   -30.36566292576   +    0.43       -0.83    4.4    1.38s
  7   -33.32045205425        0.47       -1.89    4.1    1.33s
  8   -33.33178228790       -1.95       -2.04    2.1    1.09s
  9   -33.33471887939       -2.53       -2.13    2.5    1.12s
 10   -33.33638885270       -2.78       -2.63    1.8    976ms
 11   -33.33686119870       -3.33       -2.98    3.6    1.15s
 12   -33.33687868977       -4.76      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.